In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


Checkpoint function

In [12]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

Data load & pre-processing function

In [13]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

Performance function

In [14]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

Model function

In [15]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

Create model in class version

In [16]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

Create model in function version

In [17]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

Define data & Hyper-parameter

In [18]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

Define model & optimizer & writer

In [19]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

Restore checkpoint & start train or test phrase

In [20]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer("C:/Users/johnk/logs/")
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        #checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.8697, train_loss: 2.17792225, train_accuracy: 0.4453, test_Accuracy: 0.2762
Epoch: [ 0] [    1/  468] time: 1.1529, train_loss: 2.12804317, train_accuracy: 0.5391, test_Accuracy: 0.4664
Epoch: [ 0] [    2/  468] time: 1.4362, train_loss: 2.07937574, train_accuracy: 0.6250, test_Accuracy: 0.5529
Epoch: [ 0] [    3/  468] time: 1.8151, train_loss: 1.94070399, train_accuracy: 0.6797, test_Accuracy: 0.5780
Epoch: [ 0] [    4/  468] time: 2.1473, train_loss: 1.90273726, train_accuracy: 0.6172, test_Accuracy: 0.5999
Epoch: [ 0] [    5/  468] time: 2.4255, train_loss: 1.84396505, train_accuracy: 0.6172, test_Accuracy: 0.6366
Epoch: [ 0] [    6/  468] time: 2.6978, train_loss: 1.74618113, train_accuracy: 0.6953, test_Accuracy: 0.6789
Epoch: [ 0] [    7/  468] time: 2.9691, train_loss: 1.58543158, train_accuracy: 0.7344, test_Accuracy: 0.7123
Epoch: [ 0] [    8/  468] time: 3.1855,

Epoch: [ 0] [   74/  468] time: 21.4606, train_loss: 0.28299642, train_accuracy: 0.9141, test_Accuracy: 0.9081
Epoch: [ 0] [   75/  468] time: 21.7338, train_loss: 0.28525314, train_accuracy: 0.9297, test_Accuracy: 0.9099
Epoch: [ 0] [   76/  468] time: 21.9832, train_loss: 0.21250872, train_accuracy: 0.9375, test_Accuracy: 0.9122
Epoch: [ 0] [   77/  468] time: 22.2215, train_loss: 0.28291544, train_accuracy: 0.9375, test_Accuracy: 0.9143
Epoch: [ 0] [   78/  468] time: 22.4150, train_loss: 0.26147002, train_accuracy: 0.9297, test_Accuracy: 0.9132
Epoch: [ 0] [   79/  468] time: 22.6215, train_loss: 0.41156316, train_accuracy: 0.8750, test_Accuracy: 0.9078
Epoch: [ 0] [   80/  468] time: 22.8788, train_loss: 0.16662607, train_accuracy: 0.9609, test_Accuracy: 0.9020
Epoch: [ 0] [   81/  468] time: 23.1012, train_loss: 0.33549279, train_accuracy: 0.8984, test_Accuracy: 0.9017
Epoch: [ 0] [   82/  468] time: 23.2937, train_loss: 0.36000314, train_accuracy: 0.8750, test_Accuracy: 0.9069
E

Epoch: [ 0] [  148/  468] time: 41.3414, train_loss: 0.29848921, train_accuracy: 0.8828, test_Accuracy: 0.9278
Epoch: [ 0] [  149/  468] time: 41.5967, train_loss: 0.19619012, train_accuracy: 0.9297, test_Accuracy: 0.9274
Epoch: [ 0] [  150/  468] time: 41.8201, train_loss: 0.30451334, train_accuracy: 0.8828, test_Accuracy: 0.9286
Epoch: [ 0] [  151/  468] time: 42.0714, train_loss: 0.20935744, train_accuracy: 0.9375, test_Accuracy: 0.9310
Epoch: [ 0] [  152/  468] time: 42.3487, train_loss: 0.15454452, train_accuracy: 0.9531, test_Accuracy: 0.9307
Epoch: [ 0] [  153/  468] time: 42.6359, train_loss: 0.23406774, train_accuracy: 0.9453, test_Accuracy: 0.9321
Epoch: [ 0] [  154/  468] time: 42.9391, train_loss: 0.30557573, train_accuracy: 0.9062, test_Accuracy: 0.9340
Epoch: [ 0] [  155/  468] time: 43.2153, train_loss: 0.29395044, train_accuracy: 0.9141, test_Accuracy: 0.9331
Epoch: [ 0] [  156/  468] time: 43.5574, train_loss: 0.20850745, train_accuracy: 0.9531, test_Accuracy: 0.9331
E

Epoch: [ 0] [  222/  468] time: 58.9871, train_loss: 0.23060857, train_accuracy: 0.9609, test_Accuracy: 0.9460
Epoch: [ 0] [  223/  468] time: 59.2265, train_loss: 0.15088940, train_accuracy: 0.9531, test_Accuracy: 0.9465
Epoch: [ 0] [  224/  468] time: 59.4359, train_loss: 0.21902458, train_accuracy: 0.9219, test_Accuracy: 0.9461
Epoch: [ 0] [  225/  468] time: 59.6633, train_loss: 0.17375629, train_accuracy: 0.9375, test_Accuracy: 0.9446
Epoch: [ 0] [  226/  468] time: 59.8488, train_loss: 0.19273934, train_accuracy: 0.9531, test_Accuracy: 0.9435
Epoch: [ 0] [  227/  468] time: 60.1041, train_loss: 0.16092417, train_accuracy: 0.9531, test_Accuracy: 0.9429
Epoch: [ 0] [  228/  468] time: 60.3624, train_loss: 0.33969146, train_accuracy: 0.8828, test_Accuracy: 0.9457
Epoch: [ 0] [  229/  468] time: 60.5589, train_loss: 0.17907362, train_accuracy: 0.9453, test_Accuracy: 0.9471
Epoch: [ 0] [  230/  468] time: 60.7454, train_loss: 0.17428577, train_accuracy: 0.9609, test_Accuracy: 0.9473
E

Epoch: [ 0] [  297/  468] time: 76.8812, train_loss: 0.14680554, train_accuracy: 0.9609, test_Accuracy: 0.9477
Epoch: [ 0] [  298/  468] time: 77.0827, train_loss: 0.26523644, train_accuracy: 0.9062, test_Accuracy: 0.9466
Epoch: [ 0] [  299/  468] time: 77.2632, train_loss: 0.25170010, train_accuracy: 0.9141, test_Accuracy: 0.9477
Epoch: [ 0] [  300/  468] time: 77.4587, train_loss: 0.14829814, train_accuracy: 0.9531, test_Accuracy: 0.9478
Epoch: [ 0] [  301/  468] time: 77.6432, train_loss: 0.11188789, train_accuracy: 0.9609, test_Accuracy: 0.9473
Epoch: [ 0] [  302/  468] time: 77.8606, train_loss: 0.18273529, train_accuracy: 0.9609, test_Accuracy: 0.9468
Epoch: [ 0] [  303/  468] time: 78.0611, train_loss: 0.14878413, train_accuracy: 0.9688, test_Accuracy: 0.9461
Epoch: [ 0] [  304/  468] time: 78.2675, train_loss: 0.11155885, train_accuracy: 0.9688, test_Accuracy: 0.9452
Epoch: [ 0] [  305/  468] time: 78.4530, train_loss: 0.16542596, train_accuracy: 0.9375, test_Accuracy: 0.9470
E

Epoch: [ 0] [  371/  468] time: 92.7936, train_loss: 0.09485647, train_accuracy: 0.9688, test_Accuracy: 0.9556
Epoch: [ 0] [  372/  468] time: 92.9782, train_loss: 0.28323895, train_accuracy: 0.9141, test_Accuracy: 0.9559
Epoch: [ 0] [  373/  468] time: 93.2065, train_loss: 0.10054491, train_accuracy: 0.9844, test_Accuracy: 0.9560
Epoch: [ 0] [  374/  468] time: 93.3950, train_loss: 0.22528046, train_accuracy: 0.9531, test_Accuracy: 0.9558
Epoch: [ 0] [  375/  468] time: 93.6055, train_loss: 0.16875760, train_accuracy: 0.9375, test_Accuracy: 0.9551
Epoch: [ 0] [  376/  468] time: 93.7990, train_loss: 0.08081643, train_accuracy: 0.9688, test_Accuracy: 0.9559
Epoch: [ 0] [  377/  468] time: 94.0503, train_loss: 0.17386270, train_accuracy: 0.9375, test_Accuracy: 0.9562
Epoch: [ 0] [  378/  468] time: 94.2697, train_loss: 0.26071954, train_accuracy: 0.9297, test_Accuracy: 0.9580
Epoch: [ 0] [  379/  468] time: 94.4552, train_loss: 0.13331875, train_accuracy: 0.9531, test_Accuracy: 0.9597
E

Epoch: [ 0] [  445/  468] time: 108.8088, train_loss: 0.10715853, train_accuracy: 0.9609, test_Accuracy: 0.9595
Epoch: [ 0] [  446/  468] time: 109.0003, train_loss: 0.10455222, train_accuracy: 0.9688, test_Accuracy: 0.9601
Epoch: [ 0] [  447/  468] time: 109.2227, train_loss: 0.23711674, train_accuracy: 0.9219, test_Accuracy: 0.9611
Epoch: [ 0] [  448/  468] time: 109.4411, train_loss: 0.11654493, train_accuracy: 0.9688, test_Accuracy: 0.9607
Epoch: [ 0] [  449/  468] time: 109.6775, train_loss: 0.10470136, train_accuracy: 0.9609, test_Accuracy: 0.9612
Epoch: [ 0] [  450/  468] time: 109.9188, train_loss: 0.08291066, train_accuracy: 0.9688, test_Accuracy: 0.9603
Epoch: [ 0] [  451/  468] time: 110.1233, train_loss: 0.17913207, train_accuracy: 0.9453, test_Accuracy: 0.9593
Epoch: [ 0] [  452/  468] time: 110.3766, train_loss: 0.14755565, train_accuracy: 0.9375, test_Accuracy: 0.9585
Epoch: [ 0] [  453/  468] time: 110.5671, train_loss: 0.11011402, train_accuracy: 0.9531, test_Accuracy:

In [ ]:
Test accuracy = 96.10% sigmoid 보다 RELU 에서 높게 나왔다.